In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('tops.csv')

In [2]:
import re
df['top'] = df['top'].apply(lambda x: re.sub(r'Drucksache ([0-9]+)/([0-9]+)', '', x))
df['top'] = df['top'].apply(lambda x: re.sub(r'(Befragung der Bundesregierung|Befragung|Fragestunde)', '', x))

from tqdm.notebook import tqdm
from nltk import word_tokenize

indexes = []
for i, x in tqdm(enumerate(df['top'])):
    tok = word_tokenize(x, language='german')
    if len(tok) > 6:
        indexes.append(i)

df = df.loc[indexes]
df.index = np.arange(df.shape[0])

0it [00:00, ?it/s]

In [3]:
df 

,top,top_id
0,a) Erste Beratung des von der Bundesregierung ...,1766
1,Deutschland auf zukünftige Pandemien besser vo...,408
2,5 Erste Beratung des von der Bundesregierung e...,27
3,Fortsetzung der Beteiligung bewaffneter deutsc...,1070
4,Erste Beratung des von den Fraktionen der CDU/...,898
...,...,...
1578,Aktuelle Stunde Haltung der Bundesregierung zu...,99
1579,Reform der Nachrichtendienste – Lehren aus dem...,1465
1580,Den sozialen Zusammenhalt während der Corona-K...,390
1581,Regionale Ernährungssysteme stärken Mehr Bildu...,1467


In [4]:
import tensorflow_hub as hub
import tensorflow_text
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
enc = embed(df['top']).numpy()

In [14]:
%matplotlib widget
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
c = AgglomerativeClustering(n_clusters=None, distance_threshold=0.75, affinity='cosine', linkage='average').fit_predict(enc)
print(np.unique(c))

XX = PCA(n_components=2).fit_transform(enc)
sns.scatterplot(x=XX[:, 0], y=XX[:, 1], hue=c)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [15]:
def cosine_similarity(vector_1, vector_2):
    return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1) * np.linalg.norm(vector_2))

def nearest_neighbor(v, candidates, k=1):
    indices = np.argsort([cosine_similarity(v, row) for row in candidates])[-k:]
    distances = [cosine_similarity(v, row) for row in candidates[indices]]
    return indices, distances

In [16]:
%matplotlib widget
sentence = 'ökologische Landwirtschaft'
e = embed([sentence]).numpy()[0]
neighbor_indx, distances = nearest_neighbor(e, enc, k=15)
print(df.loc[neighbor_indx]['top'].values)
plt.plot(distances)

['Agrarpolitischer Bericht der Bundesregierung 2019 Die Freisetzungsrichtlinie 2001/18/EG in ihrer Regelungsschärfe auch für neue Gentechnik beibehalten – Regulierung im Einklang mit dem Vorsorgeprinzip auch in Zukunft sichern hier:  Stellungnahme gegenüber der Bundesregierung gemäß Artikel 23 Absatz 2 des Grundgesetzes Landwirtschaft und Ernährung zukunftsfähig machen Schutz für bäuerliche Betriebe vor übergroßer Marktmacht'
 'Nationale Bioökonomiestrategie Nationale Bioökonomiestrategie der Bundesregierung SMART gestalten'
 'Haltung der Bundesregierung zur Nichtberücksichtigung der Bekämpfung von Klimakrise und des Artensterbens bei der Reform der Gemeinsamen Agrarpolitik (GAP)'
 'Dürre bekämpfen, Land und Städte widerstandsfähig aufstellen, in Klimaschutz investieren Unabhängigkeit und Innovation zur Krisenprävention in der Landwirtschaft'
 'Aus BioNTech-Erfolg lernen – Aktionsprogramm für den Gentechnik-Standort Deutschland vorlegen Chancen neuer Züchtungsmethoden erkennen – Für ei

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
latent_df = pd.read_csv('latent_space_all_20210811-2045.csv')
latent_df.index = latent_df.original_index
speech_df = pd.read_csv('rede_fraktion_top_ids.csv')

In [22]:
latent_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,fraktion,original_index
original_index,,,,,,,,,,,,,,,,,,,,,
0,0.086756,1.807908,-0.266271,1.082400,-1.213600,0.733679,0.228878,0.401669,-1.430276,0.370377,...,1.676991,1.159472,-1.345534,1.536518,0.227625,1.617102,-1.158034,2.204447,AfD,0
1,0.823400,1.661364,-0.140042,1.526995,-0.924080,0.333504,0.428154,-0.075828,-1.434303,0.827563,...,1.388907,1.173643,-1.649867,1.188291,0.117923,1.617807,-1.403038,2.112486,AfD,1
2,0.956412,0.483700,0.799958,1.436107,-1.087051,0.142430,1.329722,0.152277,-1.310064,1.441114,...,1.564915,1.253670,-1.154172,0.780806,1.349375,1.927302,-1.369963,1.463834,FDP,2
3,0.831288,1.666577,0.774181,1.274794,-1.162960,1.173674,0.651291,0.573322,-1.380169,0.397182,...,0.851646,1.505938,-1.465844,0.860317,0.948575,1.328112,-1.418897,0.940394,FDP,3
4,0.120297,1.881211,0.223084,1.001981,-1.001433,0.452178,0.605708,0.047914,-1.644455,0.595681,...,1.418988,1.238720,-1.125582,1.174178,0.556236,1.595117,-1.279481,1.788599,AfD,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20485,0.880959,0.033619,0.609298,1.396020,-1.437692,1.091345,1.384456,0.543570,-1.420133,1.445506,...,2.099916,1.066726,-1.677894,0.841271,0.953802,1.747307,-1.666726,0.859340,SPD,20485
20486,0.626187,-0.078336,0.435979,1.563795,-1.211587,0.130484,1.959904,-0.734202,-1.941073,1.619269,...,0.516340,1.387355,-1.371423,0.131650,1.006752,2.104631,-1.457125,2.061025,FDP,20486
20487,1.229690,1.173339,1.734614,1.179070,-1.875781,1.209673,0.925909,0.551052,-1.682665,0.883499,...,0.769581,1.861776,-1.580238,-0.037818,1.482101,0.814146,-1.777475,0.135623,Linke,20487


In [49]:
def get_matching_speeches(phrase, k=30):
    e = embed([phrase]).numpy()[0]
    neighbor_indx, distances = nearest_neighbor(e, enc, k=k)
    
    print(df.loc[neighbor_indx]['top'].values)
    
    top_ids = df.loc[neighbor_indx]['top_id'].values
    indx = speech_df[speech_df['top_id'].isin(top_ids)].index
    sel = latent_df.loc[indx]
    return sel.values[:, :25].astype(np.float), sel['fraktion']

m = {"AfD": 4, "B90": 5, "Union": 1, "Linke": 0, "FDP": 3, "SPD": 2, "fraktionslos":6}

In [129]:
%matplotlib widget
X, labels = get_matching_speeches('Marktwirtschaft', k=10)
XX = PCA(n_components=2).fit_transform(X)

#from sklearn.manifold import TSNE
#XX = TSNE(n_components=2, perplexity=150).fit_transform(X)

palette = [ "#FF00FF", "#000000", "#EE0000", "#EEEE00", "#0000FF", "#00FF00", "#CCCCCC"]
sns.scatterplot(x=XX[:, 0], y=XX[:, 1], hue=[m[x] for x in labels], palette=palette[:len(set(labels))])

['Einzelplan 09 Bundesministerium für Wirtschaft und Energie'
 'Chaos bei den Überbrückungshilfen beenden, Rückkehr zu marktwirtschaftlichen Prinzipien einleiten'
 'Faire digitale Märkte – Wettbewerb und Datenschutz sicherstellen'
 'Fair Play in der digitalen Wirtschaft herstellen Faire digitale Märkte – Wettbewerb und Datenschutz sicherstellen'
 'Aktuelle Stunde Wohnungsmarkt als Spielball der Politik – Haltung der Bundesregierung zu Freiheit und Eigentum im Wohnungsmarkt'
 'Einfach frei leben – Staatssozialismus verhindern, Demokratie und Wohlstand retten Green Deal zum Wohle der deutschen Wirtschaft beenden Soziale Marktwirtschaft für die Zukunft bewahren – Ökonomische Resilienz stärken'
 'Smart Germany – Deutschland als Leitmarkt für Industrie 4.0 etablieren'
 'Vorfahrt für die Marktwirtschaft – Einführung einer Beteiligungsbremse'
 'Wirtschaftliche Belebung mit marktwirtschaftlichen Prinzipien Deutsche Importwirtschaft angesichts der aktuellen Krise entlasten – Nachteile bei Einfu

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>